# Running an simulaulation model of a queueing system without priorities

The simulation model of a queueing system supports various types of input and service distributions, any number of channels. It is possible to specify the maximum length of the queue. By default, it is an infinite queue. In Kendall's notation: GI/G/n/r and GI/G/n

To run the simulation model, it requires 4 steps:
- creating an instance of the simulation model
- specifying the source distribution
- specifying the service distribution
- running the model


#### Import the QueueingSystemSimulator class and create an instance of the system. 
When creating it, specify the number of channels *n* and the maximum queue length *r*. If not specified, the queue is considered unlimited by default.

In [2]:
from most_queue.sim.queueing_systems.base import QsSim

n = 3
r = 100

qs = QsSim(n, buffer=r)

#### Setting the input stream.
The *set_sources* method takes two parameters:
- distribution parameters
- type of distribution

For exponential distribution, you only need to pass one parameter - the intensity of the incoming flow. For most distributions, parameters are specified as a list.

#### Supported types of distributions
| Distribution Type | Type | Parameters |
| --- | --- | --- |
| Exponential | 'M' | mu
| Hyper-exponential 2nd order | 'N' | [y1, mu1, mu2]
| Gamma distribution | 'Gamma' | [mu, alpha]
| Erlang | 'E'| [r, mu]
| Cox 2nd order | 'C' | [y1, mu1, mu2]
| Pareto | 'Pa'| [alpha, K]
| Deterministic | 'D' | [b]
| Uniform | 'Uniform'| [mean, half_interval]
| Normal | 'Norm'| [mean, standard_deviation]

In [3]:
l = 1.0
qs.set_sources(l, 'M')

#### Set the service time distribution. Calculate the service intensity through the utilization coefficient and the number of channels

In [4]:
ro = 0.8  # коэффициент загрузки
mu = l / (ro * n)  # интенсивность обслуживания
qs.set_servers(mu, 'M')

#### To run the simulation, you need to call the *run* method and pass the number of jobs

In [5]:
num_of_jobs = 1000000
qs.run(num_of_jobs)

Start simulation


Job served:    | 1/100 [00:00<00:12,  7.69it/s]10000/1000000:   1%|          | 1/100 [00:00<00:12,  7.69it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:13,  7.30it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:13,  7.30it/s]20000/1000000:   3%|▎         | 3/100 [00:00<00:13,  7.19it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:13,  7.19it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:13,  7.15it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:13,  7.15it/s]40000/1000000:   5%|▌         | 5/100 [00:00<00:13,  7.17it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:13,  7.17it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:12,  7.24it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:12,  7.24it/s]60000/1000000:   7%|▋         | 7/100 [00:00<00:12,  7.24it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:12,  7.24it/s]70000/1000000:   8%|▊         | 8/100 [00:01<00:12,  7.38it/s]80000/1000000:   8%|▊         | 8/100 [00:01<00:12,  7.38it/s]80000/1000000:   9%|▉  

Simulation is finished



#### After completing the simsimulation, you can obtain the initial moments of waiting and staying times, as well as the probabilities of system states.

In [6]:
print("Time spent ", qs.time_spent)
v_sim = qs.v
w_sim = qs.w
p_sim = qs.p
print(v_sim)

Time spent  14.560699937000003
[5.064919439019046, 46.809095067801465, 629.6575561703125]


#### Compare the results of simuimulation with the results of numerical calculation for the model M/M/n/r

In [7]:
from most_queue.theory.queueing_systems.fifo.mmnr import MMnrCalc
from most_queue.general.tables import times_print, probs_print

mmnr = MMnrCalc(l, mu, n, r)
w = mmnr.get_w()
times_print(w_sim, w)

Initial moments of waiting time in the system
       #       |      Num      |      Sim      
---------------------------------------------
       1       |    2.5888     |    2.6628     
       2       |     20.71     |    22.423     
       3       |    248.52     |    290.63     



#### Check for correctness of simulation results with the results of calculation for the model M/D/n

In [8]:
from most_queue.theory.queueing_systems.fifo.m_d_n import MDn

qs = QsSim(n)

qs.set_sources(l, 'M')
qs.set_servers(1.0 / mu, 'D')

qs.run(num_of_jobs)

mdn = MDn(l, 1 / mu, n)
p_ch = mdn.calc_p()
p_im = qs.get_p()

print("Time spent ", qs.time_spent)

probs_print(p_im, p_ch, 10)

Start simulation


Job served:    | 1/100 [00:00<00:12,  8.15it/s]10000/1000000:   1%|          | 1/100 [00:00<00:12,  8.15it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:11,  8.40it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:11,  8.40it/s]20000/1000000:   3%|▎         | 3/100 [00:00<00:11,  8.48it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:11,  8.48it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:11,  8.51it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:11,  8.51it/s]40000/1000000:   5%|▌         | 5/100 [00:00<00:11,  8.49it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:11,  8.49it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:11,  8.37it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:11,  8.37it/s]60000/1000000:   7%|▋         | 7/100 [00:00<00:11,  8.30it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:11,  8.30it/s]70000/1000000:   8%|▊         | 8/100 [00:00<00:11,  8.28it/s]80000/1000000:   8%|▊         | 8/100 [00:00<00:11,  8.28it/s]80000/1000000:   9%|▉  

Simulation is finished

Time spent  13.10835874
------------------------------------
      Probabilities of states       
------------------------------------
 #  |      Num      |      Sim      
------------------------------------
 0  |   0.049841    |   0.050035    
 1  |    0.13286    |    0.13329    
 2  |    0.18476    |    0.18445    
 3  |    0.18195    |    0.18156    
 4  |    0.14595    |    0.14588    
 5  |    0.10403    |    0.10383    
 6  |   0.069852    |   0.069735    
 7  |   0.045764    |   0.045714    
 8  |   0.029764    |    0.02969    
 9  |   0.019336    |   0.019352    
------------------------------------

